# Functional Test 3.2.3 - L2STS Smart NICs

This Jupyter notebook will allow you to create VMs on different sites and worker nodes consistent with requirements for test 3.2.3 for testing Smart NIC with L2STS service.


## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## Step 3: Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [ ]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the Test Slice

The following creates three nodes with Smart NICs in two selected sites. Orchestrator will pick the appropriate worker nodes, if multiple workers of the same type are present (fastnet or slownet) select them explicitly by modifying the code below. If you are unsure, the generated ads for each site ([in JSON format](https://github.com/fabric-testbed/aggregate-ads/tree/main/JSON)) can help.

Available NIC types are:
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

**You should try different NIC types**

**The code to create the slice will auto-refresh until the slice is created or it fails**

In [1]:
from datetime import datetime
from dateutil import tz

name1='Node1'
nic1_name='SmartNIC1'

name2='Node2'
nic2_name='SmartNIC2'

name3='Node3'
nic3_name='SmartNIC3'

# set NIC type for both NICs, remember workers have one type or the other
nic_type1='NIC_ConnectX_5'
nic_type2='NIC_ConnectX_5'
nic_type3='NIC_ConnectX_5'

site1='TACC' # site being tested
site2='SALT' # known good site

worker1 = None
worker2 = None
worker3 = None

# worker selection normally is commented out so it is automatic - uncomment to select specific worker
# if more than one worker of fastnet or slownet type is present
#worker1=f'{site1.lower()}-w1.fabric-testbed.net'
#worker2=f'{site2.lower()}-w1.fabric-testbed.net'
#worker3=f'{site2.lower()}-w1.fabric-testbed.net'

network_name='l2-STS'

cores=10
ram=20
disk=50
slice_name=f"Slice Test 3.2.3-SmartNIC across {site1}/{site2} on {datetime.now()}"

In [ ]:
try:
    #Create Slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Node1 goes to site1
    # Nodes2 and 3 go to site2
    
    # Node1
    node1 = slice.add_node(name=name1, site=site1, host=worker1)
    iface1 = node1.add_component(model=nic_type1, name=nic1_name).get_interfaces()[0]

    # Node2
    node2 = slice.add_node(name=name2, site=site2, host=worker2)
    iface2 = node2.add_component(model=nic_type2, name=nic2_name).get_interfaces()[0]
    
    # Node3
    node3 = slice.add_node(name=name3, site=site2, host=worker3)
    iface3 = node3.add_component(model=nic_type3, name=nic3_name).get_interfaces()[0]
    
    # Network
    net1 = slice.add_l2network(name=network_name, interfaces=[iface1, iface2, iface3], type='L2STS')
 
    # Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Step 5: Observe the Slice's Attributes

### Print the slice 

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node List

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node Details

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Interfaces

###### You should see 6 interfaces (3 cards, 2 ports).

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

### Smart NIC PCI Devices

Run the command <code>lspci</code> to see your Smart NIC PCI device(s).

View node1's Smart NIC - you should see 2 of either
- `Ethernet controller: Mellanox Technologies MT28908 Family [ConnectX-6]` 
- `Ethernet controller: Mellanox Technologies MT27800 Family [ConnectX-5]`

In [ ]:
command = "sudo dnf install -q -y pciutils && lspci | grep Mellanox"
try:
    node1 = slice.get_node(name=name1)
    stdout, stderr = node1.execute(command)
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

## Step 6: Configure interfaces, test reachability

##  Configure IP Addresses

### Pick a Subnet

Create a subnet and list of available IP addresses. You can use either IPv4 or IPv6 subnets and addresses.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Optionally, use the `node.execute()` method to show the results of adding the IP address.

In [ ]:
try:
    node1 = slice.get_node(name=name1)        
    node1_iface = node1.get_interface(network_name=network_name) 
    node1_addr = available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

### Configure Node2

Repeat the steps to add the next available IP to the second node.

In [ ]:
try:
    node2 = slice.get_node(name=name2)        
    node2_iface = node2.get_interface(network_name=network_name)  
    node2_addr = available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=subnet)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

### Configure Node3

Repeat the steps to add the next available IP to the third node.

In [ ]:
try:
    node3 = slice.get_node(name=name1)        
    node3_iface = node1.get_interface(network_name=network_name) 
    node3_addr = available_ips.pop(0)
    node3_iface.ip_addr_add(addr=node3_addr, subnet=subnet)
    
    stdout, stderr = node3.execute(f'ip addr show {node3_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

### Test reachability

Test ping between interfaces, observe successful output.


In [ ]:
try:
    node1 = slice.get_node(name=name1)        

    stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    node1 = slice.get_node(name=name1)        

    stdout, stderr = node1.execute(f'ping -c 5 {node3_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    node2 = slice.get_node(name=name2)        

    stdout, stderr = node2.execute(f'ping -c 5 {node3_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

## Step 7: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")